# Calculating with the type system

Taken from Andy Ferris's talk at JuliaCon 2018

In [9]:
# Binary representation at the type level - no run-time data!

abstract type Bit; end

struct Zero <: Bit; end
struct One <: Bit; end

In [10]:
# OR and AND for two Bits

Base.:|(::Zero, ::Zero) = Zero()
Base.:|(::Zero, ::One)  = One()
Base.:|(::One,  ::Zero) = One()
Base.:|(::One,  ::One)  = One()

Base.:&(::Zero, ::Zero) = Zero()
Base.:&(::Zero, ::One)  = Zero()
Base.:&(::One,  ::Zero) = Zero()
Base.:&(::One,  ::One)  = One()

In [11]:
# 8 Bits make a Byte

struct Byte{Bit1 <: Bit, Bit2 <: Bit, Bit3 <: Bit, Bit4 <: Bit, Bit5 <: Bit, Bit6 <: Bit, Bit7 <: Bit, Bit8 <: Bit}
    bit1::Bit1
    bit2::Bit2
    bit3::Bit3
    bit4::Bit4
    bit5::Bit5
    bit6::Bit6
    bit7::Bit7
    bit8::Bit8
end

In [12]:
# Bitwise OR and AND on Bytes

function Base.:|(byte1::Byte, byte2::Byte)
    return Byte(byte1.bit1 | byte2.bit1,
                byte1.bit2 | byte2.bit2,
                byte1.bit3 | byte2.bit3,
                byte1.bit4 | byte2.bit4,
                byte1.bit5 | byte2.bit5,
                byte1.bit6 | byte2.bit6,
                byte1.bit7 | byte2.bit7,
                byte1.bit8 | byte2.bit8)
end

function Base.:&(byte1::Byte, byte2::Byte)
    return Byte(byte1.bit1 & byte2.bit1,
                byte1.bit2 & byte2.bit2,
                byte1.bit3 & byte2.bit3,
                byte1.bit4 & byte2.bit4,
                byte1.bit5 & byte2.bit5,
                byte1.bit6 & byte2.bit6,
                byte1.bit7 & byte2.bit7,
                byte1.bit8 & byte2.bit8)
end

We can now perform almost arbitrary logic on bits and bytes! (For arbitrary we need a NAND for example)

In [13]:
byte1 = Byte(Zero(), Zero(), One(),  Zero(), Zero(), One(), One(),  One())
byte2 = Byte(Zero(), One(),  Zero(), Zero(), Zero(), One(), Zero(), One())

Byte{Zero,One,Zero,Zero,Zero,One,Zero,One}(Zero(), One(), Zero(), Zero(), Zero(), One(), Zero(), One())

In [14]:
byte1 | byte2

Byte{Zero,One,One,Zero,Zero,One,One,One}(Zero(), One(), One(), Zero(), Zero(), One(), One(), One())

In [15]:
byte1 & byte2

Byte{Zero,Zero,Zero,Zero,Zero,One,Zero,One}(Zero(), Zero(), Zero(), Zero(), Zero(), One(), Zero(), One())

In [16]:
# Let's see if type inference is doing the computation at compile-time

@code_typed byte1 | byte2

CodeInfo(
│╻ getproperty4 1 ─     (Base.getfield)(byte1, :bit1)::Zero
││  │       (Base.getfield)(byte2, :bit1)::Zero
││  │       (Base.getfield)(byte1, :bit2)::Zero
││  │       (Base.getfield)(byte2, :bit2)::One
││  │       (Base.getfield)(byte1, :bit3)::One
││  │       (Base.getfield)(byte2, :bit3)::Zero
││  │       (Base.getfield)(byte1, :bit4)::Zero
││  │       (Base.getfield)(byte2, :bit4)::Zero
││  │       (Base.getfield)(byte1, :bit5)::Zero
││  │       (Base.getfield)(byte2, :bit5)::Zero
││  │       (Base.getfield)(byte1, :bit6)::One
││  │       (Base.getfield)(byte2, :bit6)::One
││  │       (Base.getfield)(byte1, :bit7)::One
││  │       (Base.getfield)(byte2, :bit7)::Zero
││  │       (Base.getfield)(byte1, :bit8)::One
││  │       (Base.getfield)(byte2, :bit8)::One
│   └──     return $(QuoteNode(Byte{Zero,One,One,Zero,Zero,One,One,One}(Zero(), One(), One(), Zero(), Zero(), One(), One(), One())))
) => Byte{Zero,One,One,Zero,Zero,One,One,One}

### Turing Complete

To make this Turing-complete, we need some way of creating "memory". One could use more `struct`s to build a heirarchy of words and pages of increasing number of bits, complete with a system of pointers and so-on!

Alternatively, one can use the built-in `Tuple` type which can accept an arbitrary number of fields of different types. In either case, it is possible to perform just about any computation purely in the type domain.